# Setup

In [ ]:
import logging
from lisa.utils import setup_logging
setup_logging(level=logging.CRITICAL)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

hv.renderer('bokeh').theme = built_in_themes['dark_minimal']
pio.templates.default = "plotly"
pio.templates.default = "plotly_dark"

color_cycle = hv.Cycle(['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'])

opts.defaults(
    opts.Curve(tools=['hover'], width=600, height=500, show_grid=True, color=color_cycle, muted_alpha=0),
    opts.Table(bgcolor='black')
)

BENCHMARK_PATH = '/home/kajpuc01/power/pixel6/geekbench/'

def trim_number(x):
    if x > 1000000000:
        return f"{round(x / 1000000000, 3)}B"
    if x > 1000000:
        return f"{round(x / 1000000, 3)}M"
    if x > 10000:
        return f"{round(x / 1000, 2)}k"
    if x < 0.01:
        return f"{round(x * 1000000, 2)}μ"
    return str(x)

def format_percentage(vals, perc, pvals, pval_threshold=0.02):
    result = round(perc, 2).astype(str).apply(lambda s: f"({'' if s.startswith('-') or (s == '0.0') else '+'}{s}%)").to_frame()
    result['vals'] = vals.apply(lambda x: trim_number(x))
    result['pvals'] = pvals
    result['pval_marker'] = pvals.apply(lambda x: "* " if x < pval_threshold else "")
    result['value'] = result['vals'] + " " + result['pval_marker'] + result['value']
    return result['value']

def plot_gmean_bars(df, x='stat', y='value', facet_col='metric', facet_col_wrap=3, title='', width=800, height=600, gmean_round=1, include_columns=[], table_sort=None, order_cluster=False, sort_ascending=False, include_total=False, debug=False):
    shown_clusters = clusters if not include_total else clusters_total
    
    if not 'unit' in df.columns:
        df['unit'] = 'x'
    if not 'metric' in df.columns:
        df['metric'] = 'gmean'
        
    if debug:
        print('df')
        display(df)
        
        
    # compute percentage differences
    stats_perc = Stats(df, ref_group={'wa_path': a_wa_path}, value_col=y, agg_cols=['iteration'], stats={'gmean': sp.stats.gmean}).df
    # re-add stub a_wa_path
    stats_perc_values_temp = stats_perc.query("wa_path == @b_wa_path")
    stats_perc_values_temp['wa_path'] = a_wa_path
    stats_perc_values_temp['value'] = 0
    # re-combine a df with percentage differences
    stats_perc_values = pd.concat([stats_perc_values_temp, stats_perc])
    stats_perc_values['order_kernel'] = stats_perc_values['wa_path'].map(lambda x: wa_paths.index(x))
    
    if debug:
        print('stats_perc_values')
        display(stats_perc_values)
    
    sort_list = ['metric']
    
    if order_cluster:
        sort_list.append('order_cluster')
        stats_perc_values['order_cluster'] = stats_perc_values['cluster'].map(lambda x: shown_clusters.index(x))
        
    sort_list.append('order_kernel')

    # split into dfs with percentages and pvalues
    stats_perc_pvalues = stats_perc_values.query("stat == 'ks2samp_test'").sort_values(by=sort_list).reset_index(drop=True)
    stats_perc_values = stats_perc_values.query("stat == 'gmean'").sort_values(by=sort_list).reset_index(drop=True)

    # compute absolute gmeans
    gmeans = Stats(df, agg_cols=['iteration'], stats={'gmean': sp.stats.gmean, 'std': None, 'sem': None}).df
    if gmean_round > 0:
        gmeans['value'] = round(gmeans['value'], gmean_round)
    gmeans['order_kernel'] = gmeans['wa_path'].map(lambda x: wa_paths.index(x))
    
    if order_cluster:
        gmeans['order_cluster'] = gmeans['cluster'].map(lambda x: shown_clusters.index(x))
        
    if debug:
        display(stats_perc_pvalues)

    gmeans_mean = gmeans.query("stat == 'gmean'").sort_values(by=sort_list).reset_index(drop=True)
    if debug:
        print(sort_list)
        print('gmeans')
        display(gmeans)
        
    data_table_cols = [col for col in gmeans_mean.columns if col in (['wa_path', 'value', 'test_name', 'variable', 'metric', 'chan_name', 'comm'] + include_columns)]
    data_table = gmeans_mean[data_table_cols].rename(columns={'wa_path':'kernel'})
    data_table['perc_diff'] = stats_perc_values['value'].map(lambda x: str(round(x, 2)) + '%')
    data_table['value'] = data_table['value'].apply(lambda x: trim_number(x))
    if table_sort is not None:
        data_table = data_table.sort_values(by=table_sort)
    ptable(data_table)
        
    # plot bars
    fig = px.bar(gmeans_mean, x=x, y=y, color='wa_path', facet_col=facet_col, facet_col_wrap=facet_col_wrap, barmode='group', text=format_percentage(gmeans_mean['value'], stats_perc_values['value'], stats_perc_pvalues['value']), title=title, width=width, height=height)
    fig.update_traces(textposition='outside')
    fig.update_yaxes(matches=None)
    if sort_ascending:
        fig.update_xaxes(categoryorder='total ascending')
    fig.show()

    return data_table
    
def ptable(df):
    print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False, floatfmt=".3f"))

def trim_wa_path(path):
    return path[10:-7]
    #return path[10:]

## Runs

In [ ]:
benchmark_name_a = 'geekbench_baseline_3_0812'
benchmark_name_b = 'geekbench_eas_lock_3_1301'
benchmark_name_c = 'geekbench_ufc_patched_v4_3_2001'
benchmark_name_d = 'geekbench_ufc_feec_all_cpus_3_3001'
benchmark_name_e = 'geekbench_ufc_feec_all_cpus_fits_3_3001'

wa_output_a = WAOutput(BENCHMARK_PATH + benchmark_name_a)
wa_output_b = WAOutput(BENCHMARK_PATH + benchmark_name_b)
wa_output_c = WAOutput(BENCHMARK_PATH + benchmark_name_c)
wa_output_d = WAOutput(BENCHMARK_PATH + benchmark_name_d)
wa_output_e = WAOutput(BENCHMARK_PATH + benchmark_name_e)

df_a = wa_output_a['results'].df
df_b = wa_output_b['results'].df
df_c = wa_output_c['results'].df
df_d = wa_output_d['results'].df
df_e = wa_output_e['results'].df

a_kernel = df_a['kernel'][0]
b_kernel = df_b['kernel'][0]
c_kernel = df_c['kernel'][0]
d_kernel = df_d['kernel'][0]
e_kernel = df_e['kernel'][0]

a_wa_path = trim_wa_path(df_a['wa_path'][0])
b_wa_path = trim_wa_path(df_b['wa_path'][0])
c_wa_path = trim_wa_path(df_c['wa_path'][0])
d_wa_path = trim_wa_path(df_d['wa_path'][0])
e_wa_path = trim_wa_path(df_e['wa_path'][0])
wa_paths = [a_wa_path, b_wa_path, c_wa_path, d_wa_path, e_wa_path]

df = pd.concat([df_a, df_b, df_c, df_d, df_e])
df = df.drop(columns=['scaled from(%)'])
df['wa_path'] = trim_wa_path(df['wa_path'].str)
df_perf = df[df['metric'].str.contains('perf')].reset_index(drop=True).query("value != 0")
df_perf['metric'] = df_perf['metric'].str[7:]
df = df[~df['metric'].str.contains('perf')].reset_index(drop=True).query("value != 0")

clusters = ['little', 'mid', 'big']
clusters_total = ['little', 'mid', 'big', 'total']

print(benchmark_name_a, benchmark_name_b, benchmark_name_c, benchmark_name_d, benchmark_name_e)

print(wa_paths)
print(a_kernel, b_kernel, c_kernel, d_kernel, e_kernel)

# Benchmark scores

## Line plot

In [ ]:
metrics = ['multicore_score', 'score']
ds = hv.Dataset(df, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('metric', values=metrics)], 'value')
layout = ds.select(metric=metrics).to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('metric').opts(shared_axes=False, title='Benchmark score per-iteration').cols(2)
layout.opts(
    opts.Curve(height=600, width=900),
)
layout

## Bar plot

In [ ]:
benchmark_scores = plot_gmean_bars(df.query("metric in @metrics"), x='stat', y='value', facet_col='metric', facet_col_wrap=3, title='gmean score', width=1600, height=600)

# Overutilized

In [ ]:
try:
    overutils = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/overutilized.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/overutilized.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/overutilized.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/overutilized.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/overutilized.pqt'),
    ]

    overutilized_combined = pd.concat(overutils)
    overutilized_combined['wa_path'] = trim_wa_path(overutilized_combined['wa_path'].str)
    overutilized_combined['time'] = round(overutilized_combined['time'], 2)
    overutilized_combined['total_time'] = round(overutilized_combined['total_time'], 2)
    
    overutilized_mean = overutilized_combined.groupby(['wa_path']).agg(lambda x: series_mean(x)).reset_index().rename(columns={'wa_path':'kernel'})
    overutilized_mean['metric'] = 'overutilized'
    overutilized_mean = overutilized_mean[['metric', 'kernel', 'time', 'total_time', 'percentage']]
    overutilized_mean['percentage'] = round(overutilized_mean['percentage'], 2)
    overutilized_mean['time'] = round(overutilized_mean['time'], 2)
    overutilized_mean['total_time'] = round(overutilized_mean['total_time'], 2)
    ptable(overutilized_mean)
    
except FileNotFoundError:
    print('overutilized.pqt not found.')

## Line plot

In [ ]:
fig = px.line(overutilized_combined, x='iteration', y='percentage', color='wa_path', height=600, title='Overutilized percentage per-iteration')
fig.show()

# Perf

## Line plot

In [ ]:
metrics = ['cpu-migrations', 'context-switches', 'stalled-cycles-backend', 'page-faults', 'major-faults', 'cache-misses', 'instructions', 'cpu-cycles', 'cpu-clock']
ds = hv.Dataset(df_perf, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('metric', values=metrics)], 'value')
layout = ds.select(metric=metrics).to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('metric').opts(shared_axes=False, title='Perf counters').cols(3)
layout.opts(
    opts.Curve(width=600, height=340),
    opts.Overlay(legend_position='bottom'),
)
layout

## Bar plot

In [ ]:
metrics = ['cpu-migrations', 'context-switches', 'stalled-cycles-backend', 'page-faults', 'major-faults', 'minor-faults', 'cache-misses', 'instructions', 'cpu-cycles', 'cpu-clock']
plot_gmean_bars(df_perf.query("metric in @metrics"), x='stat', y='value', facet_col='metric', facet_col_wrap=5, title='gmean perf counters', width=1900, height=900)

# Idle residency

In [ ]:
try:
    idle_residency_times_a = pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/idle_residency.pqt')
    idle_residency_times_b = pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/idle_residency.pqt')
    idle_residency_times_c = pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/idle_residency.pqt')
    idle_residency_times_d = pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/idle_residency.pqt')
    
    idle_residency_times_combined = pd.concat([idle_residency_times_a, idle_residency_times_b, idle_residency_times_c, idle_residency_times_d])
    idle_residency_times_combined['wa_path'] = trim_wa_path(idle_residency_times_combined['wa_path'].str)
    
    display(idle_residency_times_combined)
except FileNotFoundError:
    print('idle_residency.pqt not found.')

## Bar plot

In [ ]:
residency_data = idle_residency_times_combined.groupby(['wa_path', 'cluster', 'idle_state'], sort=False).mean().reset_index()[['wa_path', 'cluster', 'idle_state', 'time']]
residency_data['time'] = round(residency_data['time'], 2)

fig = px.bar(residency_data, x='idle_state', y='time', color='wa_path', facet_col='cluster', barmode='group', text=residency_data['time'],
             width=1800, height=600, title='Idle state residencies')
fig.update_traces(textposition='outside')
fig.show()

# Idle misses

In [ ]:
try:
    cpu_idle_a = pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/cpu_idle.pqt')
    cpu_idle_b = pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/cpu_idle.pqt')
    cpu_idle_c = pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/cpu_idle.pqt')
    cpu_idle_d = pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/cpu_idle.pqt')
    cpu_idle_a_wakeups = len(cpu_idle_a.query("state == 4294967295"))
    cpu_idle_b_wakeups = len(cpu_idle_b.query("state == 4294967295"))
    cpu_idle_c_wakeups = len(cpu_idle_c.query("state == 4294967295"))
    cpu_idle_d_wakeups = len(cpu_idle_d.query("state == 4294967295"))
except FileNotFoundError:
    print('cpu_idle.pqt not found.')

try:
    cpu_idle_miss_a = pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/cpu_idle_miss_counts.pqt')
    cpu_idle_miss_a['count_perc'] = round(cpu_idle_miss_a['count'] / cpu_idle_a_wakeups * 100, 3)
    cpu_idle_miss_b = pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/cpu_idle_miss_counts.pqt')
    cpu_idle_miss_b['count_perc'] = round(cpu_idle_miss_b['count'] / cpu_idle_b_wakeups * 100, 3)
    cpu_idle_miss_c = pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/cpu_idle_miss_counts.pqt')
    cpu_idle_miss_c['count_perc'] = round(cpu_idle_miss_c['count'] / cpu_idle_c_wakeups * 100, 3)
    cpu_idle_miss_d = pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/cpu_idle_miss_counts.pqt')
    cpu_idle_miss_d['count_perc'] = round(cpu_idle_miss_d['count'] / cpu_idle_d_wakeups * 100, 3)
    
    cpu_idle_miss_combined = pd.concat([cpu_idle_miss_a, cpu_idle_miss_b, cpu_idle_miss_c, cpu_idle_miss_d])
    cpu_idle_miss_combined['type'] = cpu_idle_miss_combined['below'].replace(0, 'too deep').replace(1, 'too shallow')
    cpu_idle_miss_combined['wa_path'] = cpu_idle_miss_combined['wa_path'].str[10:-7]
    
    display(cpu_idle_miss_combined)
    
    a_miss_percentage = round(cpu_idle_miss_a['count'].sum() / cpu_idle_a_wakeups * 100, 3)
    b_miss_percentage = round(cpu_idle_miss_b['count'].sum() / cpu_idle_b_wakeups * 100, 3)
    c_miss_percentage = round(cpu_idle_miss_c['count'].sum() / cpu_idle_c_wakeups * 100, 3)
    d_miss_percentage = round(cpu_idle_miss_d['count'].sum() / cpu_idle_d_wakeups * 100, 3)
    print(f"{a_miss_percentage}% {a_wa_path} vs {b_miss_percentage}% {b_wa_path}")
    print(f"{c_miss_percentage}% {c_wa_path} vs {d_miss_percentage}% {d_wa_path}")
except FileNotFoundError:
    print('cpu_idle_miss_counts.pqt not found.')

In [ ]:
ptable(cpu_idle_miss_combined.groupby(['wa_path', 'type']).sum().reset_index()[['wa_path', 'type', 'count_perc']])

## Bar plot

In [ ]:
fig = px.bar(cpu_idle_miss_combined, x='type', y='count_perc', color='wa_path', facet_col='cluster', barmode='group', text=cpu_idle_miss_combined['count_perc'], width=1800, height=600, title='CPUIdle misses as percentage of all wakeups')
fig.show()

# Power usage

In [ ]:
try:
    pixel6_emeters = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/pixel6_emeter.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/pixel6_emeter.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/pixel6_emeter.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/pixel6_emeter.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/pixel6_emeter.pqt'),
    ]
    
    pixel6_emeter_means = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/pixel6_emeter_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/pixel6_emeter_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/pixel6_emeter_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/pixel6_emeter_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/pixel6_emeter_mean.pqt'),
    ]
    
    pixel6_emeters_combined = pd.concat(pixel6_emeters)
    pixel6_emeters_combined['wa_path'] = trim_wa_path(pixel6_emeters_combined['wa_path'].str)
    pixel6_emeter_means_combined = pd.concat(pixel6_emeter_means)
    pixel6_emeter_means_combined['total_power'] = pixel6_emeter_means_combined[['little_power', 'mid_power', 'big_power']].sum(axis=1)
    pixel6_emeter_means_combined['wa_path'] = trim_wa_path(pixel6_emeter_means_combined['wa_path'].str)
    
    pixel6_emeter_melt = pd.melt(pixel6_emeter_means_combined, id_vars=['iteration', 'wa_path', 'kernel'], value_vars=['little_power', 'mid_power', 'big_power', 'total_power'])
    pixel6_emeter_melt['cluster'] = pixel6_emeter_melt['chan_name'].str[:-6]
except FileNotFoundError:
    print('pixel6_emeter.pqt not found.')

In [ ]:
pixel6_emeters_combined.groupby(['wa_path', 'iteration']).sum().reset_index()

## Line plot

In [ ]:
fig = px.line(pixel6_emeter_melt, x='iteration', y='value', color='wa_path', facet_col='cluster', height=600, title='Mean power usage across iterations [mW]')
fig.show()

## Bar plot

In [ ]:
power_usage = plot_gmean_bars(pixel6_emeter_melt, x='chan_name', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean power usage [mW]', width=1800, height=600, order_cluster=True, include_total=True)

# Energy estimate

In [ ]:
try:
    energy_estimate_means = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/energy_estimate_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/energy_estimate_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/energy_estimate_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/energy_estimate_mean.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/energy_estimate_mean.pqt'),
    ]
    
    energy_estimate_means_combined = pd.concat(energy_estimate_means)
    energy_estimate_means_combined['wa_path'] = energy_estimate_means_combined['wa_path'].str[20:-7]
    energy_estimate_melt = pd.melt(energy_estimate_means_combined, id_vars=['iteration', 'wa_path'], value_vars=['little', 'mid', 'big', 'total']).rename(columns={'variable':'cluster'})

except FileNotFoundError:
    print('energy_estimate_mean.pqt not found.')

## Line plot

In [ ]:
fig = px.line(energy_estimate_melt, x='iteration', y='value', color='wa_path', facet_col='cluster', height=600, title='Mean energy estimate across iterations [bW]')
fig.show()

## Bar plot

In [ ]:
energy_estimate = plot_gmean_bars(energy_estimate_melt, x='cluster', y='value', facet_col='metric', facet_col_wrap=5, title='Gmean energy estimate [bW]', width=1900, height=600, include_columns=['cluster'], order_cluster=True, include_total=True)

# Thermal

In [ ]:
try:
    thermals = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/thermal.pqt').groupby(['iteration', 'kernel', 'wa_path']).agg(lambda x: series_mean(x)).reset_index(),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/thermal.pqt').groupby(['iteration', 'kernel', 'wa_path']).agg(lambda x: series_mean(x)).reset_index(),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/thermal.pqt').groupby(['iteration', 'kernel', 'wa_path']).agg(lambda x: series_mean(x)).reset_index(),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/thermal.pqt').groupby(['iteration', 'kernel', 'wa_path']).agg(lambda x: series_mean(x)).reset_index(),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/thermal.pqt').groupby(['iteration', 'kernel', 'wa_path']).agg(lambda x: series_mean(x)).reset_index()
    ]
    
    for i in range(len(thermals)):
        for col in [c for c in thermals[i].columns if c not in ['time', 'iteration', 'kernel', 'wa_path']]:
            thermals[i][col] = thermals[i][col] / 1000
        thermals[i] = round(thermals[i], 2)
        thermals[i]['wa_path'] = thermals[i]['wa_path'].str[20:-7]

    thermal_combined = pd.concat(thermals)
    thermal_melt = pd.melt(thermal_combined, id_vars=['iteration', 'wa_path', 'kernel'], value_vars=['little', 'mid', 'big']).rename(columns={'variable':'cluster'})
except FileNotFoundError:
    print('thermal.pqt not found.')

## Line plot

In [ ]:
ds = hv.Dataset(thermal_melt, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('cluster', values=clusters)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster temperature across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='temperature'),
)
layout

## Bar plot

In [ ]:
thermal = plot_gmean_bars(thermal_melt, x='cluster', y='value', facet_col='metric', facet_col_wrap=2, title='Gmean temperature', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# Frequency

In [ ]:
freqs_means = [
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/freqs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/freqs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/freqs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/freqs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/freqs_mean.pqt'),
]
    
freqs_mean_combined = pd.concat(freqs_means)
freqs_mean_combined['wa_path'] = trim_wa_path(freqs_mean_combined['wa_path'].str)
freqs_mean_combined['unit'] = 'MHz'
freqs_mean_combined['metric'] = 'frequency'

freqs_mean_combined['order'] = freqs_mean_combined['cluster'].replace('little', 0).replace('mid', 1).replace('big', 2)
freqs_mean_combined = freqs_mean_combined.sort_values(by=['iteration', 'order']).rename(columns={'frequency':'value'})

display(freqs_mean_combined.head())

## Line plot

In [ ]:
ds = hv.Dataset(freqs_mean_combined, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('cluster', values=clusters)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster frequency across iterations')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='MHz'),
)
layout

## Bar plot

In [ ]:
frequency = plot_gmean_bars(freqs_mean_combined, x='metric', y='value', facet_col='cluster', facet_col_wrap=3, title='Gmean frequency per cluster', width=1800, height=600, order_cluster=True, include_columns=['cluster'])

# CFS signals

In [ ]:
cfs_signals = [
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/sched_pelt_cfs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/sched_pelt_cfs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/sched_pelt_cfs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/sched_pelt_cfs_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/sched_pelt_cfs_mean.pqt'),
]

cfs_signals_combined = pd.concat(cfs_signals)
cfs_signals_combined['wa_path'] = trim_wa_path(cfs_signals_combined['wa_path'].str)
cfs_signals_combined['kernel'] = 'android-mainline-5.18'
cfs_signals_melt = pd.melt(cfs_signals_combined, id_vars=['iteration', 'wa_path', 'kernel', 'cluster'], value_vars=['util', 'load'])

## Line plot

In [ ]:
signals = ['util', 'load']
ds = hv.Dataset(cfs_signals_combined, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('cluster', values=clusters)], signals)
layout = hv.Layout([ds.to(hv.Curve, 'iteration', signal).overlay('wa_path').opts(legend_position='bottom').layout('cluster').opts(title='Mean cluster ' + signal) for signal in signals]).cols(1)
layout.opts(
    opts.Curve(width=800, height=400),
)
layout

## Bar plot

In [ ]:
cfs_signals = plot_gmean_bars(cfs_signals_melt, x='cluster', y='value', facet_col='variable', facet_col_wrap=1, title='Gmean cfs signals', width=1900, height=1000, order_cluster=True, include_columns=['cluster'])

# Task wakeup latency

In [ ]:
wakeup_latency_means = [
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/wakeup_latency_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/wakeup_latency_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/wakeup_latency_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/wakeup_latency_mean.pqt'),
    pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/wakeup_latency_mean.pqt'),
]

wakeup_latency_mean_combined = pd.concat(wakeup_latency_means).rename(columns={'wakeup_latency':'value'})
wakeup_latency_mean_combined['wa_path'] = trim_wa_path(wakeup_latency_mean_combined['wa_path'].str)
wakeup_latency_mean_combined['order'] = wakeup_latency_mean_combined['wa_path'].map(lambda x: wa_paths.index(x))
wakeup_latency_mean_combined['unit'] = 'x'

wakeup_latencies = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/wakeup_latency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/wakeup_latency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/wakeup_latency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/wakeup_latency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/wakeup_latency.pqt'),
]
    
wakeup_latency_combined = pd.concat(wakeup_latencies)
wakeup_latency_combined['wa_path'] = trim_wa_path(wakeup_latency_combined['wa_path'].str)
wakeup_latency_combined['order'] = wakeup_latency_combined['wa_path'].map(lambda x: wa_paths.index(x))
wakeup_latency_combined['cluster'] = wakeup_latency_combined['cpu'].copy().apply(lambda c: 'little' if c < 4 else 'big' if c > 5 else 'mid')
wakeup_latency_combined['order_cluster'] = wakeup_latency_combined['cluster'].map(lambda x: clusters.index(x))
wakeup_latency_combined['target_cluster'] = wakeup_latency_combined['target_cpu'].copy().apply(lambda c: 'little' if c < 4 else 'big' if c > 5 else 'mid')
wakeup_latency_combined['order_target_cluster'] = wakeup_latency_combined['target_cluster'].map(lambda x: clusters.index(x))
wakeup_latency_combined

## Line plot

In [ ]:
fig = px.line(wakeup_latency_mean_combined, x='iteration', y='value', color='wa_path', facet_col='comm', facet_col_wrap=3, height=600, title='Task wakeup latencies across iterations')
fig.show()

## Bar plot

In [ ]:
wakeup_latency = plot_gmean_bars(wakeup_latency_mean_combined, x='metric', y='value', facet_col='comm', facet_col_wrap=3, title='Gmean task wakeup latency', table_sort=['comm', 'kernel'], gmean_round=0, width=1900, height=600)

## Quantiles

In [ ]:
wakeup_latency_quantiles = wakeup_latency_combined.groupby(['comm', 'wa_path', 'iteration']).quantile([0.9, 0.95, 0.99]).reset_index()[['comm', 'wa_path', 'level_3', 'iteration', 'wakeup_latency', 'order']].rename(columns={'level_3':'quantile'}).sort_values(by=['comm', 'order'])
wakeup_latency_quantiles

In [ ]:
plot_gmean_bars(wakeup_latency_quantiles.rename(columns={'wakeup_latency':'value'}), x='quantile', y='value', facet_col='comm', facet_col_wrap=1, title='Gmean latency quantile', width=1900, include_columns=['quantile'], table_sort=['quantile', 'comm'], height=1000, gmean_round=0)

## Per-execution cluster

In [ ]:
latency_cluster = wakeup_latency_combined.groupby(['comm', 'wa_path', 'cluster']).mean().reset_index().sort_values(by=['comm', 'order_cluster', 'order'])[['comm', 'wa_path', 'cluster', 'wakeup_latency']]
fig = px.bar(latency_cluster, x='cluster', y='wakeup_latency', color='wa_path', facet_col='comm', barmode='group', facet_col_wrap=1, width=2000, height=1300, text=latency_cluster['wakeup_latency'].apply(lambda x: trim_number(x)), title='Mean task wakeup latency per cluster')
fig.update_traces(textposition='outside')
latency_cluster['wakeup_latency'] = latency_cluster['wakeup_latency'].apply(lambda x: trim_number(x))
ptable(latency_cluster)
fig.show()

## Per-target cluster

In [ ]:
latency_target_cluster = wakeup_latency_combined.groupby(['comm', 'wa_path', 'target_cluster']).mean().reset_index().sort_values(by=['comm', 'order_target_cluster', 'order'])[['comm', 'wa_path', 'target_cluster', 'wakeup_latency']]
fig = px.bar(latency_target_cluster, x='target_cluster', y='wakeup_latency', color='wa_path', facet_col='comm', barmode='group', facet_col_wrap=1, width=2000, height=1300, text=latency_target_cluster['wakeup_latency'].apply(lambda x: trim_number(x)), title='Mean task wakeup latency per target_cluster')
fig.update_traces(textposition='outside')
latency_target_cluster['wakeup_latency'] = latency_target_cluster['wakeup_latency'].apply(lambda x: trim_number(x))
ptable(latency_target_cluster)
fig.show()

# Wakeup latency - cgroups

In [ ]:
wakeup_latencies_cgroup = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/wakeup_latency_cgroup.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/wakeup_latency_cgroup.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/wakeup_latency_cgroup.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/wakeup_latency_cgroup.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/wakeup_latency_cgroup.pqt'),
]

wakeup_latency_cgroup_combined = pd.concat(wakeup_latencies_cgroup).rename(columns={'wakeup_latency':'value'})
wakeup_latency_cgroup_combined['wa_path'] = trim_wa_path(wakeup_latency_cgroup_combined['wa_path'].str)
wakeup_latency_cgroup_combined['order'] = wakeup_latency_cgroup_combined['wa_path'].map(lambda x: wa_paths.index(x))

wakeup_latency_cgroup_mean = wakeup_latency_cgroup_combined.groupby(["wa_path", "cgroup", "iteration", "order"]).agg(lambda x: series_mean(x)).reset_index().sort_values(by=["order", "cgroup", "iteration"])[['wa_path', 'cgroup', 'iteration', 'value', 'order']]
wakeup_latency_cgroup_mean

## Line plot

In [ ]:
fig = px.line(wakeup_latency_cgroup_mean, x='iteration', y='value', color='wa_path', facet_col='cgroup', facet_col_wrap=3, height=600, title='cgroup wakeup latencies across iterations')
fig.update_yaxes(matches=None)
fig.show()

## Bar plot

In [ ]:
plot_gmean_bars(wakeup_latency_cgroup_mean, x='metric', y='value', facet_col='cgroup', title='Gmean task wakeup latency per-cgroup', include_columns=['cgroup'], table_sort=['cgroup'], gmean_round=0, width=1800, height=600)

## Quantiles

In [ ]:
wakeup_latency_cgroup_quantiles = wakeup_latency_cgroup_mean.groupby(['cgroup', 'wa_path', 'iteration']).quantile([0.9, 0.95, 0.99]).reset_index()[['cgroup', 'wa_path', 'level_3', 'iteration', 'value', 'order']].rename(columns={'level_3':'quantile'}).sort_values(by=['cgroup', 'order'])
wakeup_latency_cgroup_quantiles

In [ ]:
plot_gmean_bars(wakeup_latency_cgroup_quantiles, x='quantile', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean latency quantile per-cgroup', include_columns=['cgroup', 'quantile'], table_sort=['quantile', 'cgroup'], width=1900, height=1400, gmean_round=0)

# Task CPU residency

In [ ]:
try:
    task_residency_totals = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/tasks_residency_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/tasks_residency_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/tasks_residency_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/tasks_residency_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/tasks_residency_total.pqt'),
    ]

    task_residencies = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/tasks_residency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/tasks_residency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/tasks_residency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/tasks_residency.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/tasks_residency.pqt'),
    ]

    
    cpus = ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0']
    task_residency_total_combined = pd.concat(task_residency_totals).rename(columns={'Total':'total'})
    task_residency_total_combined['wa_path'] = trim_wa_path(task_residency_total_combined['wa_path'].str)
    task_residency_total_melt = pd.melt(task_residency_total_combined, id_vars=['iteration', 'wa_path', 'kernel'], value_vars=cpus).rename(columns={'variable':'cluster'})
    task_residency_total_cluster_melt = pd.melt(task_residency_total_combined, id_vars=['iteration', 'wa_path', 'kernel'], value_vars=clusters_total).rename(columns={'cpu':'cluster'})

    task_residencies_combined = pd.concat(task_residencies).rename(columns={'Total':'total'}).query("comm == 'AsyncTask #1'")
    task_residencies_combined['wa_path'] = trim_wa_path(task_residencies_combined['wa_path'].str)
    task_residencies_combined_cluster_melt = pd.melt(task_residencies_combined, id_vars=['iteration', 'wa_path', 'kernel', 'comm'], value_vars=clusters_total).rename(columns={'cpu':'cluster'})
    
    display(task_residency_total_combined)
except FileNotFoundError:
    print('thermal.pqt not found.')

## Clusters - Line plot

In [ ]:
ds = hv.Dataset(task_residency_total_cluster_melt, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('cluster', values=clusters_total)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cluster').cols(4).opts(title='Mean cluster CPU residency')
layout.opts(
    opts.Curve(width=500, height=600, ylabel='time'),
)
layout

## Clusters - Bar plot

In [ ]:
plot_gmean_bars(task_residency_total_cluster_melt, x='cluster', y='value', facet_col='metric', facet_col_wrap=2, title='Gmean cluster CPU residency', include_columns=['cluster'], width=1800, height=600, order_cluster=True, include_total=True)

## Clusters - Per-task bar plot

In [ ]:
task_cpu_residency = plot_gmean_bars(task_residencies_combined_cluster_melt, x='cluster', y='value', facet_col='comm', facet_col_wrap=2, title='Gmean cluster task CPU residency', include_columns=['cluster'], width=1800, height=600, order_cluster=True, include_total=True)

## CPUs - Line plot

In [ ]:
ds = hv.Dataset(task_residency_total_melt, ['iteration', hv.Dimension('wa_path', values=wa_paths), hv.Dimension('cpu', values=cpus)], 'value')
layout = ds.to(hv.Curve, 'iteration', 'value').overlay('wa_path').opts(legend_position='bottom').layout('cpu').cols(3).opts(title='Mean CPU residency')
layout.opts(
    opts.Curve(width=600, height=600, ylabel='time'),
)
layout

## CPUs - Bar plot

In [ ]:
plot_gmean_bars(task_residency_total_melt, x='cpu', y='value', facet_col='metric', facet_col_wrap=2, title='Gmean CPU residency', width=1900, height=700)

# CPU residency - cgroups

In [ ]:
try:
    cgroup_residency_totals = [
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_a + '/task_residency_cgroup_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_b + '/task_residency_cgroup_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_c + '/task_residency_cgroup_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_d + '/task_residency_cgroup_total.pqt'),
        pd.read_parquet(BENCHMARK_PATH + benchmark_name_e + '/task_residency_cgroup_total.pqt'),
    ]
    
    cpus = ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0']
    cgroup_residency_total_combined = pd.concat(cgroup_residency_totals).rename(columns={'Total':'total'})[['wa_path', 'cgroup', 'iteration', 'total', 'little', 'mid', 'big', '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0']]
    cgroup_residency_total_combined['wa_path'] = trim_wa_path(cgroup_residency_total_combined['wa_path'].str)
    cgroup_residency_total_melt = pd.melt(cgroup_residency_total_combined, id_vars=['iteration', 'wa_path', 'cgroup'], value_vars=cpus).rename(columns={'variable':'cluster'})
    cgroup_residency_total_cluster_melt = pd.melt(cgroup_residency_total_combined, id_vars=['iteration', 'wa_path', 'cgroup'], value_vars=clusters_total).rename(columns={'cpu':'cluster'})
    
    display(cgroup_residency_total_combined)
except FileNotFoundError:
    print('task_residency_cgroup_total.pqt not found.')

## Clusters - Bar plot

In [ ]:
plot_gmean_bars(cgroup_residency_total_cluster_melt, x='cluster', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean cluster CPU residency per-cgroup', width=1800, height=1100, include_columns=['cgroup', 'cluster'], table_sort=['cgroup', 'cluster'], order_cluster=True, include_total=True)

## CPUs - Bar plot

In [ ]:
plot_gmean_bars(cgroup_residency_total_melt, x='cpu', y='value', facet_col='cgroup', facet_col_wrap=1, title='Gmean cgroup CPU residency', width=1800, height=1100, include_columns=['cgroup', 'cpu'])

# Summary - TLDR

In [ ]:
summary_parts = []

try:
    summary_benchmark_scores = benchmark_scores.copy()
    summary_benchmark_scores['perc_diff'] = summary_benchmark_scores['perc_diff'].apply(lambda s: f"({s})")
    summary_benchmark_scores['value'] = summary_benchmark_scores['value'] + " " + summary_benchmark_scores['perc_diff']
    summary_benchmark_scores = summary_benchmark_scores.pivot(values='value', columns='kernel', index='metric').reset_index()[['metric'] + wa_paths]
    summary_parts.append(summary_benchmark_scores)
except NameError:
    pass

try:
    summary_power_usage = power_usage.copy().query("chan_name == 'total_power'")
    summary_power_usage['perc_diff'] = summary_power_usage['perc_diff'].apply(lambda s: f"({s})")
    summary_power_usage['value'] = summary_power_usage['value'] + " " + summary_power_usage['perc_diff']
    summary_power_usage = summary_power_usage.pivot(values='value', columns='kernel', index='chan_name').reset_index().rename(columns={'chan_name':'metric'})[['metric'] + wa_paths]
    summary_parts.append(summary_power_usage)
except NameError:
    pass

try:
    summary_ou = overutilized_mean.copy()
    summary_ou['percentage'] = summary_ou['percentage'].apply(lambda x: f"{x}%")
    summary_ou = summary_ou.pivot(values='percentage', columns='kernel', index='metric').reset_index()[['metric'] + wa_paths]
    summary_parts.append(summary_ou)
except NameError:
    pass

try:
    summary_thermal = thermal.copy()
    summary_thermal['perc_diff'] = summary_thermal['perc_diff'].apply(lambda s: f"({s})")
    summary_thermal['value'] = summary_thermal['value'] + " " + summary_thermal['perc_diff']
    summary_thermal = summary_thermal.pivot(values='value', columns='kernel', index='cluster').reset_index().rename(columns={'cluster':'metric'})[['metric'] + wa_paths]
    summary_thermal['metric'] = "thermal (" + summary_thermal['metric'] + ")"
    summary_parts.append(summary_thermal)
except NameError:
    pass

try:
    summary_wakeup_latency = wakeup_latency.copy()
    summary_wakeup_latency['perc_diff'] = summary_wakeup_latency['perc_diff'].apply(lambda s: f"({s})")
    summary_wakeup_latency['comm'] = "latency (" + summary_wakeup_latency['comm'] + ")"
    summary_wakeup_latency['value'] = summary_wakeup_latency['value'] + " " + summary_wakeup_latency['perc_diff']
    summary_wakeup_latency = summary_wakeup_latency.pivot(values='value', columns='kernel', index='comm').reset_index().rename(columns={'comm':'metric'})[['metric'] + wa_paths]
    summary_parts.append(summary_wakeup_latency)
except NameError:
    pass

try:
    summary_task_cpu_residency = task_cpu_residency.copy().query("cluster == 'total'")
    summary_task_cpu_residency['perc_diff'] = summary_task_cpu_residency['perc_diff'].apply(lambda s: f"({s})")
    summary_task_cpu_residency['value'] = summary_task_cpu_residency['value'] + " " + summary_task_cpu_residency['perc_diff']
    summary_task_cpu_residency = summary_task_cpu_residency.pivot(values='value', columns='kernel', index='comm').reset_index().rename(columns={'comm':'metric'})[['metric'] + wa_paths]
    summary_task_cpu_residency['metric'] = "CPU residency (" + summary_task_cpu_residency['metric'] + ")"
    summary_parts.append(summary_task_cpu_residency)
except NameError:
    pass

summary = pd.concat(summary_parts).reset_index(drop=True)

print('Geekbench 5')
ptable(summary)